Домашнее задание 30 (необходимо сделать 3 задачи из 4) на Python:
1. Найти самый старый фильм и самый новый фильм - вывести их названия по 1 шт (2 запроса)

In [70]:
# функция для получения самого старого фильма
def get_oldest_movie(tx):
    result = tx.run(
        """
        MATCH (movie:Movie)                      // находим все фильмы
        WITH MIN(movie.released) AS oldest_year  // определяем минимальный год выпуска
        MATCH (m:Movie {released: oldest_year})  // находим фильм с этим годом
        RETURN m.title AS oldest_title           // возвращаем его название
        LIMIT 1                                  // ограничиваем до одного результата
        """
    )
    return result.single()["oldest_title"]       # извлекаем значение из результата

# функция для получения самого нового фильма
def get_newest_movie(tx):
    result = tx.run(
        """
        MATCH (movie:Movie)                      // находим все фильмы
        WITH MAX(movie.released) AS newest_year  // определяем максимальный год выпуска
        MATCH (m:Movie {released: newest_year})  // находим фильм с этим годом
        RETURN m.title AS newest_title           // возвращаем его название
        LIMIT 1                                  // ограничиваем до одного результата
        """
    )
    return result.single()["newest_title"]       # извлекаем значение из результата

In [71]:
# открываем сессию и выполняем оба запроса
with driver.session() as session:
    oldest_title = session.execute_read(get_oldest_movie)  # получаем самый старый фильм
    newest_title = session.execute_read(get_newest_movie)  # получаем самый новый фильм
    print(f"Самый старый фильм: {oldest_title}")
    print(f"Самый новый фильм: {newest_title}")


Самый старый фильм: One Flew Over the Cuckoo's Nest
Самый новый фильм: Cloud Atlas


2. Получить среднее количество актёров на фильм 

In [72]:
# Функция для вычисления среднего количества актёров на один фильм
def avg_actors_on_movie(tx):
    result = tx.run(
        """
        MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie) // Находим пары актёр — фильм
        WITH movie, COUNT(actor) AS actor_count         // Считаем количество актёров для каждого фильма
        RETURN AVG(actor_count) AS avg_actors           // Вычисляем среднее количество актёров на фильм
        """
    )
    return result.single()["avg_actors"]                # Возвращаем значение из результата запроса


In [73]:
 # Открываем сессию
with driver.session() as session: 
    result = session.execute_read(avg_actors_on_movie)           # выполняем функцию, которая читает данные из БД в транзакции  
    print(f'Среднее количество актеров на один фильм: {result}') # выводим результат 


Среднее количество актеров на один фильм: 4.526315789473684


3. Группировка фильмов по годам и подсчёт количества фильмов в каждом году

In [74]:
# Функция для подсчета количества фильмов в каждом году
def cnt_movies_on_every_year(tx):
    result = tx.run(
        """
        MATCH (m:Movie)                                 // находим все фильмы
        WITH m.released as year, COUNT(m) as cnt_movie  // группируем по году выпуска, считаем количество фильмов
        RETURN year, cnt_movie                          // возвращаем год и количество фильмов
        ORDER BY year                                   // сортируем по возрастанию года
        """
    )
    return [record.data() for record in result]         # преобразуем результаты в список словарей


In [75]:
import pandas as pd

In [76]:
# Открываем сессию
with driver.session() as session:  
    result = session.execute_read(cnt_movies_on_every_year)  # выполняем функцию, которая читает данные из БД в транзакции
    df = pd.DataFrame(result)                                # создаём DataFrame из списка словарей
    print(df)                                                # выводим DataFrame

    year  cnt_movie
0   1975          1
1   1986          2
2   1990          1
3   1992          4
4   1993          1
5   1995          2
6   1996          3
7   1997          2
8   1998          3
9   1999          4
10  2000          3
11  2003          3
12  2004          1
13  2006          3
14  2007          1
15  2008          2
16  2009          1
17  2012          1


4*. Найти актёра с наибольшим промежутком между первым и последним фильмом

In [77]:
# Функция для поиска актера с промежутком между первым и последним фильмом
def actor_with_biggest_interval(tx):
    result = tx.run(
        """
        MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)                                  // находим актёров и фильмы, где они снимались
        WITH actor, MIN(movie.released) as first_film, MAX(movie.released) as last_film  // группируем по актеру и находим мин и макс год выпуска фильма для каждого актёра
        RETURN
            actor.name as actor_name,                                                    // имя актёра
            last_film - first_film as interval                                           // вычисляем интервал между первым и последним фильмом
        ORDER BY interval DESC                                                           // сортируем по убыванию интервала
        LIMIT 1                                                                          // берём актёра с максимальным интервалом
        """
    )
    return result.single()                                                               # возвращаем первый результат из объекта Result



In [78]:
# Открываем сессию
with driver.session() as session:
    result = session.execute_read(actor_with_biggest_interval)                                            # выполняем функцию, которая читает данные из БД в транзакции
    print(f"Актёр с наибольшим промежутком: {result['actor_name']}, интервал: {result['interval']} лет")  # выводим результат, обращаясь к полям записи по ключам


Актёр с наибольшим промежутком: Jack Nicholson, интервал: 28 лет
